In [2]:
# Import libraries
!pip install statsmodels
!pip install missingno
!pip install scikit-learn
!pip install xgboost
import numpy as np 
import seaborn as sns
import plotly.graph_objects as go
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import os
import sklearn
from scipy import stats
import xgboost as xgb

In [3]:
df = pd.read_csv('/Users/davidfrost/Documents/modern_data_analytics/assignment/data.csv')

EDA

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['label_enc'] = le.fit_transform(df['label'])

In [5]:
df.head(10)

,id,temp,humidity,wind,rain,target,label,events_count,nameday,hour,label_enc
0,2022-01-01 00:00:00,13.131834,89.350000,0.042000,0.000000,55.88125,Intermediate,0,5,0,1
1,2022-01-01 01:00:00,13.129184,88.458333,0.063333,0.000000,58.05625,High,0,5,1,0
2,2022-01-01 02:00:00,12.927130,88.291667,0.026667,0.000125,51.57500,Intermediate,0,5,2,1
3,2022-01-01 03:00:00,12.800749,88.041667,0.027083,0.000000,48.67500,Intermediate,0,5,3,1
4,2022-01-01 04:00:00,12.524946,87.666667,0.028333,0.000000,46.12500,Intermediate,0,5,4,1
5,2022-01-01 05:00:00,11.859896,88.541667,0.015417,0.000000,44.55000,Intermediate,0,5,5,1
6,2022-01-01 06:00:00,11.503234,88.833333,0.020000,0.000000,41.57500,Low,0,5,6,2
7,2022-01-01 07:00:00,11.334430,88.958333,0.037083,0.000000,41.70000,Low,0,5,7,2
8,2022-01-01 08:00:00,10.970783,90.000000,0.025833,0.000000,41.73125,Low,0,5,8,2
9,2022-01-01 09:00:00,11.365553,88.958333,0.137083,0.000000,43.00000,Low,0,5,9,2


Splitting data

In [79]:
# Extracting correct features
from sklearn.model_selection import train_test_split
x = df[['temp', 'nameday', 'hour', 'humidity', 'rain', 'wind', 'events_count']]
  
y = df['label_enc']

X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2)

In [94]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.exceptions import NotFittedError
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier

# Define numeric and categorical features
numeric_features = ['temp', 'humidity', 'rain', 'wind']
categorical_features = ['hour', 'nameday', 'events_count']


# Define transformers for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create preprocessor for numeric and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create pipeline with preprocessor and XGBoost classifier
xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(objective='multi:softmax', num_class=3))
])
# Fit the pipeline on the training data
xgb.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'humidity', 'rain',
                                                   'wind']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['hour', 'nameday',
                                                   'events_count'])])),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_byl...
                               grow_policy=None, importance_ty

In [95]:
# Make predictions on the test data
y_pred_xgb = xgb.predict(X_test)

# # Fit the OneHotEncoder transformer to the categorical features
# categorical_transformer.fit(X_train[categorical_features])

# # Get feature importances and names
# importances = rfc.named_steps['regressor'].feature_importances_
# encoded_cat_features = categorical_transformer.named_steps['onehot'].get_feature_names_out(categorical_features)
# all_feature_names = numeric_features + list(encoded_cat_features)
# feature_importances = np.zeros(len(all_feature_names))

# # Add the importances of the original numeric features
# feature_importances[:len(numeric_features)] += importances[:len(numeric_features)]

# # Combine the importances of the encoded categorical features into the original features
# feature_importances = importances[:len(numeric_features)].tolist()  # Start with the numeric features
# for i, feature_name in enumerate(categorical_features):
#     encoded_cat_importances = [
#         importances[j] for j, feat_name in enumerate(all_feature_names)
#         if feat_name.startswith(feature_name + '_')
#     ]
#     feature_importances.append(sum(encoded_cat_importances))

# # Get the names of the original features
# original_feature_names = numeric_features + categorical_features

# # Plot the feature importances
# plt.figure(figsize=(10,8))
# plt.barh(original_feature_names, feature_importances)
# plt.xlabel('Feature Importance')
# plt.ylabel('Features')
# plt.title('Random Forest Class Feature Importances')
# plt.show()

In [97]:
from sklearn.metrics import classification_report

# Generate predictions on the test set
y_pred_xgb = xgb.predict(X_test)

# Create a classification report
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.46      0.14      0.21       236
           1       0.73      0.94      0.82      1204
           2       0.52      0.17      0.26       279

    accuracy                           0.71      1719
   macro avg       0.57      0.42      0.43      1719
weighted avg       0.66      0.71      0.65      1719



In [98]:
preds = pd.DataFrame(y_pred_xgb)
preds.value_counts()

1    1555
2      93
0      71
dtype: int64

In [99]:
from sklearn.model_selection import cross_val_score, KFold
# Create a KFold object with 5 folds
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and print the results
cv_scores = cross_val_score(xgb, x, y, cv=kfold)
print("Cross-validation scores: ", cv_scores)
print("Average accuracy: {:.2f}".format(cv_scores.mean()))

Cross-validation scores:  [0.70855148 0.72134962 0.71653085 0.7363213  0.71420256]
Average accuracy: 0.72


In [106]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for grid search
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__learning_rate': [0.1, 0.01, 0.001],
    'classifier__max_depth': [3, 4, 5],
    'classifier__subsample': [0.8, 0.9],
    'classifier__max_delta_step': [1, 2],
    'classifier__tree_method': ['hist']
}

# Create the GridSearchCV object
grid_search = GridSearchCV(xgb, param_grid=param_grid, cv=3)

# Fit the GridSearchCV on the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score from the grid search
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'classifier__learning_rate': 0.1, 'classifier__max_delta_step': 1, 'classifier__max_depth': 5, 'classifier__n_estimators': 200, 'classifier__subsample': 0.8, 'classifier__tree_method': 'hist'}
Best Score: 0.7191910373926961


In [108]:
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from datetime import timedelta
content = "https://weather.com/weather/hourbyhour/l/c097b546627cdff2da1e276cb9b2731055718a5e7270d777a92857a9701c7870"
response = requests.get(content)
soup = BeautifulSoup(response.content, 'html.parser')

temp_val = soup.findAll('div', attrs={'class':'DetailsTable--field--CPpc_'})


forecast = pd.DataFrame()
forecast['temp'] = [round((int(temp_val[i].text[-3:-1])-32 ) *5/9, 1) for i in list(np.array(range(288))) if i%6 == 0]
forecast['wind'] = [float(temp_val[i].text.split(' ')[1]) for i in list(np.array(range(288))) if i%6 == 1]
forecast['wind_direction'] = [temp_val[i].text.split(' ')[0] for i in list(np.array(range(288))) if i%6 == 1]
forecast['humidity'] = [int(temp_val[i].text[-3:-1]) for i in list(np.array(range(288))) if i%6 == 2]
forecast['cloud_cover'] = [int(temp_val[i].text.replace('Cloud Cover', '')[:-1]) for i in list(np.array(range(288))) if i%6 == 4]
forecast['rain'] = [int(temp_val[i].text.replace('Rain Amount', '').replace(' in', '')) for i in list(np.array(range(288))) if i%6 == 5]
weekday = [(datetime.now()+timedelta(hours=i)).weekday() for i in range(48)]
hour_of_day = [(datetime.now()+timedelta(hours=i)).hour for i in range(48)]
forecast['nameday'] = weekday
forecast['hour'] = hour_of_day
forecast['events_count'] = '' # This value has to be included by the user. So edit this. The value now is missing, but the model running, so even if nothing is provided, it will run
forecast['tag_category'] = '' # This value has to be included by the user. So edit this


In [109]:
# Generate predictions on the test set
prediction_app = xgb.predict(forecast)

In [110]:
prediction_app

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 2, 2, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0], dtype=int32)

In [111]:
import pickle
# Saving the model to a file
filename = '/Users/davidfrost/Documents/modern_data_analytics/assignment/classifier.pkl' # add specific path
with open(filename, 'wb') as file:
    pickle.dump(xgb, file)

:(